# TO DO for next year
Code the general structure of the metaheuristics, but let them code the inner structure of the functions
move to julia 1.0
découper plus le TP

# Quick introduction to Julia

(Courtesy from Vincent Leclère)

## What is a Jupyter notebook ?

A Jupyter notebook is a document that contains
+ text
  - that can be formatted using Markdown
  - that can contains maths using $\LaTeX$
+ code
  - and we can interact with this
  
Un notebook is a succession of cells, each containing code or text. :
+ double-click to edit a cell
+ Ctrl-enter to execute a cell
+ shift-enter to execute a cell and go to the next one
+ Alt-enter to execute a cell and create a new one

You can download your project as an .ipynb file

## What is JuliaBox ?

JuliaBox is a website giving you a session with Julia and Jupyter installed. You can therefore use these softwares without installing them on your computer.

Attention, la session offert par JuliaBox dure 3h, au bout desquelles les calculs effectués sont perdus (mais pas le code). Donc, si d'un seul coup vous voyez des variables non-définies : pas d'affolement, il suffit de relancer le kernel (barre du haut) et recharger les cellules précédente.

## What is Julia ?
​
Julia is a recent programming language. It can be compared to python but desgined for scientific commputing and much faster in practice 

​
Try to execute an modify the following cells:

In [3]:
1+1

2

In [5]:
a = [0 5 10 15]
a[1]

0

In [7]:
length(a)

4

In [8]:
for i = 1:5
    println("itération ",i)
end

itération 1
itération 2
itération 3
itération 4
itération 5


In [10]:
function factorielle(n)
    res = 1
    for i=1:n
        res = res * i
    end
    return res
end

factorielle (generic function with 1 method)

In [11]:
factorielle(5)

120

# Facility Location problem considered

Consider the facility location problem with $N$ clients, $M$ facilities, among which $p$ should be chosen. Matrix $(d_{ij})$ gives the distance of facility $i$ to client $j$

\begin{eqnarray}
\min f(x,y) &=& \sum_{i=1}^N \sum_{i=1}^M d_{ij} x_{ij} \\
\text{s.c. } \sum_{j=1}^M y_j &= &p\\
\sum_{j=1}^M x_{ij} &=& 1 \text{    } i=1,\cdots ,N \\
x_{ij} &\leq & y_j \text{    } i=1,\cdots ,N; j=1,\cdots ,M \\
x_{ij} &\geq & 0 \text{    }  i=1,\cdots ,N; j=1,\cdots ,M \\
y_j & \in & \{0,1\}  \text{    }  j=1,\cdots ,M
\end{eqnarray}

# Parse instances 

The following function enables to read an instance

In [2]:
function parseInput(filename)
    file_l = open(filename)     
    lin = readline(file_l)
    nbc, nbf, nbo = map(x->parse(Int16,x), split(lin, " "))
    d = Array{Array{Int64}}(nbc)
    for ln in enumerate(eachline(file_l))
        vl = split(ln[2], " ")
        d[ln[1]] = map(x->parse(Int16,x), vl[1:nbc])
    end
    close(file_l)
    return (nbc, nbf, nbo, d)
end 

parseInput (generic function with 1 method)

# Local search:

#### Compute cost 

In [13]:
function computeClientCost(client, isOpened, nearestFac, distances)
    for fac in nearestFac[client]
        if isOpened[fac]
           return distances[client][fac] 
        end
    end
end

function computeCost(isOpened, nearestFac, distances)
    result::Int = 0
    for i in 1:length(distances)
       result += computeClientCost(i, isOpened, nearestFac, distances) 
    end
    return result
end

computeCost (generic function with 1 method)

#### Data structure used by the heuristic

In [4]:
mutable struct Ls_data 
    # Instance
    nbClients::Int
    nbFacilities::Int
    nbOpened::Int
    distances::Array{Array{Int64}}
    nearestFacilitiesOfClient::Array{Array{Int,1}}

    # Current solution
    isOpened::Array{Bool,1}
    cost::Int
    
    # best solution
    bestIsOpen::Array{Int, 1}
    bestCost::Int

    function Ls_data(nbOpened_l, distances_l)
        nbClients = length(distances_l)
        nbFacilities = length(distances_l)
        nbOpened = nbOpened_l
        distances = distances_l
        nearestFacilitiesOfClient = [sortperm(distances[i]) for i in 1:nbClients]
        
        isOpened = [j<= nbOpened for j in 1:nbFacilities]
        cost = computeCost(isOpened,nearestFacilitiesOfClient,distances)

        bestIsOpen = isOpened
        bestCost = 1e10
    
        new(nbClients, nbFacilities, nbOpened, distances, nearestFacilitiesOfClient, isOpened, cost, bestIsOpen, bestCost)
    end
end


#### Swap neighborhood (non-incremental eveluation) and local descent algorithm

In [15]:
function swap!(i::Int, j::Int, data::Ls_data)
    if data.isOpened[i] == data.isOpened[j]
        throw()
    end
    previous_cost = data.cost
    data.isOpened[i] = !data.isOpened[i];
    data.isOpened[j] = !data.isOpened[j];
    data.cost = computeCost(data.isOpened, data.nearestFacilitiesOfClient,data.distances)
    return data.cost - previous_cost
end

function firstImproving!(data::Ls_data)
    for i in 1:(data.nbFacilities - 1)
        for j in (i+1):nbFacilities
            if (data.isOpened[i] != data.isOpened[j])
                if (swap!(i,j,data)<0)
                    return true;
                else
                    swap!(i,j,data)
                end
            end
        end
    end
    return false;
end

function localDescent!(data::Ls_data)
   println("initialCost ", data.cost)
    i = 0
    while(firstImproving!(data))
        i+=1
        if (i%10 == 0)
            println(i, " iterations")
        end
    end
    println("End of Local Descent")
    println(i, "iterations")
    println("final cost:", data.cost)
end

localDescent! (generic function with 1 method)

#### Launching the Local Descent

In [16]:
filename = "./instances/pmed35.dat"
nbClients, nbFacilities, nbOpened, distances = parseInput(filename)
print("Clients:", nbClients, "\nFacilities", nbFacilities, "\nOpened:", nbOpened)

# x0 = Array{Bool,1}(nbFacilities)
x0 = [j<= nbOpened for j in 1:nbFacilities];

data = Ls_data(nbOpened,distances);
localDescent!(data)

for i in 1:data.nbClients
    if data.isOpened[i]
        print(i, " ")
    end
end


Clients:800
Facilities800
Opened:5initialCost 12983
End of Local Descent
22iterations
final cost:10400
94 114 217 305 349 

# Questions on heuristics

- The local search implemented above is not incremental. Turn it into an incremental evalutation
- Implement one of the following metaheuristic on the problem
    - Simulated annealing
    - Genetic algorithm
    - taboo search

# MILP

We are now going to see how MILP solvers can be used. Using solvers requires to use some packages. Execute the following cell only on you computer if the packages are not installed.

In [ ]:
Pkg.add("JuMP")
Pkg.add("Cbc")

In [13]:
filename = "./instances/pmed1.dat"
nbClients, nbFacilities, nbOpened, distances = parseInput(filename)
println("Clients:", nbClients, "\nFacilities", nbFacilities, "\nOpened:", nbOpened)


println(" =========== ")
println("MIP")
println(" =========== ")

using JuMP, Cbc
model = Model(solver=CbcSolver(logLevel=6))

@variable(model, y[1:nbFacilities], Bin)
@variable(model, x[1:nbClients, 1:nbFacilities], Bin)
for cl in 1:nbClients
     @constraint(model, sum(x[cl,fac] for fac in 1:nbFacilities)  == 1)
     for fac in 1:nbFacilities
         @constraint(model, x[cl,fac] <= y[fac])
     end
end
@constraint(model, sum(y[fac] for fac in 1:nbFacilities) == 
nbOpened)
@objective(model, Min, sum(x[cl,fac]*distances[cl][fac] for fac in 
1:nbFacilities, cl in 1:nbClients))
solve(model)
@show getobjectivevalue(model)                             

Clients:100
Facilities100
Opened:5
MIP
getobjectivevalue(model) = 5819.0
Cgl0006I 100 SOS (10000 members out of 10100) with 0 overlaps - too much overlap or too many others
Cgl0004I processed model has 10101 rows, 10100 columns (10100 integer (10100 of which binary)) and 30100 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 5819
Cbc0038I Before mini branch and bound, 10100 integers at bound fixed and 0 continuous
Cbc0038I Mini branch and bound did not improve solution (0.48 seconds)
Cbc0038I After 0.48 seconds - Feasibility pump exiting with objective of 5819 - took 0.01 seconds
Cbc0045I Heuristic feasibility pump took 0.00921 seconds (good)
BFeasible (0) - obj 5819 5819
Cbc0012I Integer solution of 5819 found by feasibility pump after 0 iterations and 0 nodes (0.48 seconds)
Cbc0001I Search completed - best objective 5819, took 0 iterations and 0 nodes (0.49 seconds)
Cbc0035I Maximum depth 0, 0 

5819.0

## Questions

- Remove the constraint limiting the number of facilities, but add a fixed cost of 1000 by facility opened
- Add a constraint that limits the number of clients served by a factory to 20